In [1]:
!activate PythonGPU
import numpy as np
from scipy.stats import skewnorm, skew
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error, classification_report, accuracy_score

def simulate_data(classes, n_vars, n, max_mu, max_sigma, max_skew):
    #The multivariate skew normal number generator
    def rng(mu, sigma, skew, n=1):
        k = len(mu)
        if not (k == len(sigma) and k ==len(skew)): 
            raise Exception("Mu, Sigma and Skew should be same length")

        data = np.zeros((int(n),k))

        for i in range(k):
            data[:,i] = skewnorm.rvs(skew[i], loc=mu[i], scale=sigma[i], size=int(n)) 

        return data
    
    if(np.sum(classes) != 1):
        raise Exception("Classes dont sum up to 1")
        
    n_classes = len(classes)
#     sigma = np.random.randint(1,max_sigma,n_vars)
#     skew = np.random.randint(-max_skew,max_skew,n_vars)
#     mu =  np.random.randint(-max_mu, max_mu, (n_classes, n_vars))
    
    sigma = (max_sigma * np.random.rand(1,n_vars))[0]
    skew = ((2 * max_skew  * np.random.rand(1, n_vars)) - max_skew)[0]
    mu = (2 *  max_mu * np.random.rand(n_classes, n_vars)) - max_mu
    
    n_obs_class = np.round(np.dot(classes,n))
    
    data = np.zeros((int(np.sum(n_obs_class)),n_vars+1))
    for i in range(n_classes):
        #calculate indexes
        start = int(np.sum(n_obs_class[0:i]))
        end = int(np.sum(n_obs_class[0:i+1]))
        
        #set the data
        data[start:end,0] = i
        data[start:end,1:] = rng(mu[i,:], sigma, skew, n_obs_class[i])
        
    X = data[:,1:]
    y = data[:,0]
        
    X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.33, 
    random_state=42,
    stratify=y)
    
    return X_train, X_test, y_train, y_test

In [3]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

def classify_lda(X_train, X_test, y_train, y_test, priors, plot=False):
    lda = LinearDiscriminantAnalysis(priors=priors)
    X_lda = lda.fit_transform(X_train, y_train)

    predictions = lda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("LDA Test accuracy "+ str(accuracy))
    print(predictions)

    if plot:    
        plt.xlabel('LD1')
        plt.ylabel('LD2')
        plt.scatter(
            X_lda[:,0],
            X_lda[:,1],
            c=y_train,
            cmap='Accent',
        )
        
    return {"method": "LDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": lda}

In [4]:
#Quadratic
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def classify_qda(X_train, X_test, y_train, y_test, priors):
    qda = QuadraticDiscriminantAnalysis(priors=priors)
    X_qda = qda.fit(X_train, y_train)



    predictions = qda.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("QDA Test accuracy "+ str(accuracy))

    return {"method": "QDA", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": qda}

In [5]:
from sklearn.linear_model import LogisticRegression

def classify_logit(X_train, X_test, y_train, y_test):
    clf = LogisticRegression(random_state=0, solver='lbfgs',
                             multi_class='multinomial').fit(X_train, y_train)

    predictions = clf.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print("Logistic Test accuracy "+ str(accuracy))
    
    return {"method": "Logit", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": clf}

In [6]:
#KNN
from sklearn.neighbors import KNeighborsClassifier

def classify_knn(X_train, X_test, y_train, y_test, n_neighbors):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric='euclidean')
    knn.fit(X_train, y_train)

    predictions = knn.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("KNN-"+str(n_neighbors)+" Test accuracy "+ str(accuracy))
    
    return {"method": "KNN-"+str(n_neighbors), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": knn}

In [7]:
#Naive bayes
from sklearn.naive_bayes import GaussianNB

def classify_naivebayes(X_train, X_test, y_train, y_test, priors):
    NB = GaussianNB(priors)
    NB.fit(X_train, y_train)
    
    predictions = NB.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("Naive Bayes Test accuracy "+ str(accuracy))
    
    return {"method": "Naive Bayes", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": NB}

In [8]:
#SVM
from sklearn.svm import LinearSVC

def classify_svm(X_train, X_test, y_train, y_test):
    svm = LinearSVC()
    svm.fit(X_train, y_train)
    
    predictions = svm.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    
    print("SVM Test accuracy "+ str(accuracy))
    
    return {"method": "SVM", 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": svm}

In [9]:
from tensorflow.keras import layers
from tensorflow import keras

def classify_neuralnet(X_train, X_test, y_train, y_test, n_vars, n_classes, depth=1, nodes=10, epochs=20):
    inputs = keras.Input(shape=(n_vars,), name='obs')
    x = layers.Dense(nodes, activation='relu')(inputs)
    
    if(depth>1):
        for i in range(depth-1):
            x = layers.Dense(nodes, activation='relu')(x)
            
    outputs = layers.Dense(n_classes, activation='softmax')(x)

    model = keras.Model(inputs=inputs, outputs=outputs, name='Dataset')

    display(model.summary())


    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=keras.optimizers.RMSprop(),
                  metrics=['accuracy'])

    history = model.fit(X_train, y_train,
                        batch_size=64,
                        epochs=epochs,
                        validation_split=0.2)

    predictions = model.predict(X_test)
    predictions = np.argmax(predictions, axis=1)
    print(predictions)

    accuracy = accuracy_score(y_test, predictions)
    print("Neural Network Test accuracy "+ str(accuracy))
    
    return {"method": "Net "+"-".join([str(nodes) for i in range(depth)])+ " E"+str(epochs), 
            "accuracy": accuracy, 
            "predictions":predictions,
            "model": model}

In [ ]:
from datetime import datetime
import pandas as pd
import time


np.random.seed(12345)

timelabel =  datetime.now().strftime("%H-%M-%S - %d-%m-%Y")


configs = [{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 2,
    "n": 500,
    "max_mu": 2,
    "max_sigma": 2,
    "max_skew": 1
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.05, 0.05, 0.05, 0.85],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 50,
    "n": 10000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 50,
    "n": 100000,
    "max_mu": 5,
    "max_sigma": 5,
    "max_skew": 5
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 10000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
},
{
    "classes": [0.25, 0.25, 0.25, 0.25],
    "n_vars": 5,
    "n": 100000,
    "max_mu": 1,
    "max_sigma": 10,
    "max_skew": 10
}
] 

columns = ['method', 'accuracy','predictions', "model", "config"] + list(configs[0].keys())
results = pd.DataFrame(columns=columns)
results.style.format({
    'accuracy': '{:,.3%}'.format
})



#run analysis
for i, c in enumerate(configs):
    X_train, X_test, y_train, y_test = simulate_data(c["classes"], c["n_vars"], c["n"], c["max_mu"], c["max_sigma"], c["max_skew"])
    
    lda = classify_lda(X_train, X_test, y_train, y_test, c["classes"], False)                                     
    results = results.append({**lda, **c, "config":i+1},ignore_index=True)
    
    qda = classify_qda(X_train, X_test, y_train, y_test, c["classes"])                                     
    results = results.append({**qda, **c, "config":i+1},ignore_index=True)
    
    logit = classify_logit(X_train, X_test, y_train, y_test)                                     
    results = results.append({**logit, **c, "config":i+1},ignore_index=True)
    
    for k in [5,10,50,100]:
        knn = classify_knn(X_train, X_test, y_train, y_test, k)
        results = results.append({**knn, **c, "config":i+1},ignore_index=True)

    bayes = classify_naivebayes(X_train, X_test, y_train, y_test, c["classes"])
    results = results.append({**bayes, **c, "config":i+1},ignore_index=True)
    
    svm = classify_svm(X_train, X_test, y_train, y_test)
    results = results.append({**svm, **c, "config":i+1},ignore_index=True)
    
    for n in [{"d":1,"n":len(c["classes"]), "e":25}, {"d":4,"n":30, "e":10}, {"d":4,"n":50, "e":50}]:
        neuralnet = classify_neuralnet(X_train, X_test, y_train, y_test, c["n_vars"], len(c["classes"]),  depth=n["d"], nodes=n["n"], epochs=n["e"])                                 
        results = results.append({**neuralnet, **c, "config":i+1},ignore_index=True)
    
    print("Results after config "+str(i+1)+" of "+str(len(configs)))

    results.sort_values(by='accuracy', ascending=False, inplace=True)
        
    display(results.style.format({
    'accuracy': '{:,.3%}'.format
    }))
    

    #saving results to file
    results.drop(columns=['model']).to_pickle("./results/config "+str(i+1)+" of "+str(len(configs))+" "+timelabel+".pkl")

LDA Test accuracy 0.5575757575757576
[3. 0. 0. 1. 1. 3. 0. 1. 0. 3. 2. 1. 1. 0. 0. 3. 3. 1. 2. 3. 0. 2. 1. 2.
 1. 3. 3. 2. 0. 3. 3. 1. 2. 3. 1. 1. 3. 1. 3. 1. 3. 0. 2. 2. 0. 1. 3. 2.
 3. 1. 1. 3. 0. 3. 0. 3. 3. 2. 0. 1. 0. 0. 0. 1. 3. 0. 3. 3. 2. 0. 3. 1.
 1. 1. 2. 3. 3. 0. 0. 1. 1. 3. 3. 3. 0. 0. 2. 3. 2. 0. 2. 3. 1. 3. 3. 0.
 0. 3. 2. 3. 0. 0. 0. 3. 1. 2. 0. 0. 1. 2. 1. 3. 3. 0. 0. 1. 3. 1. 3. 1.
 0. 1. 3. 3. 0. 1. 3. 1. 1. 0. 3. 0. 3. 2. 3. 1. 1. 0. 3. 1. 1. 1. 0. 2.
 1. 0. 2. 1. 1. 1. 3. 1. 1. 3. 0. 3. 1. 0. 0. 3. 1. 0. 2. 2. 1.]
QDA Test accuracy 0.5272727272727272
Logistic Test accuracy 0.5636363636363636
KNN-5 Test accuracy 0.4484848484848485
KNN-10 Test accuracy 0.4909090909090909
KNN-50 Test accuracy 0.5393939393939394
KNN-100 Test accuracy 0.4727272727272727
Naive Bayes Test accuracy 0.5333333333333333
SVM Test accuracy 0.5636363636363636


C:\Users\Tim\Anaconda3\lib\site-packages\sklearn\svm\base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Model: "Dataset"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
obs (InputLayer)             [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 12        
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 20        
Total params: 32
Trainable params: 32
Non-trainable params: 0
_________________________________________________________________


None

Train on 268 samples, validate on 67 samples
Epoch 1/25
268/268 [==============================] - 1s 5ms/sample - loss: 1.4615 - accuracy: 0.2836 - val_loss: 1.3603 - val_accuracy: 0.3284
Epoch 2/25
268/268 [==============================] - 0s 190us/sample - loss: 1.4475 - accuracy: 0.2985 - val_loss: 1.3507 - val_accuracy: 0.3284
Epoch 3/25
268/268 [==============================] - 0s 198us/sample - loss: 1.4366 - accuracy: 0.3060 - val_loss: 1.3433 - val_accuracy: 0.3433
Epoch 4/25
268/268 [==============================] - 0s 179us/sample - loss: 1.4285 - accuracy: 0.3097 - val_loss: 1.3364 - val_accuracy: 0.3433
Epoch 5/25
268/268 [==============================] - 0s 216us/sample - loss: 1.4203 - accuracy: 0.3134 - val_loss: 1.3298 - val_accuracy: 0.3582
Epoch 6/25
268/268 [==============================] - 0s 198us/sample - loss: 1.4129 - accuracy: 0.3246 - val_loss: 1.3248 - val_accuracy: 0.3582
Epoch 7/25
268/268 [==============================] - 0s 220us/sample - loss: 1.4

...

Train on 53600 samples, validate on 13400 samples
Epoch 1/25
53600/53600 [==============================] - 5s 102us/sample - loss: 1.9469 - accuracy: 0.2603 - val_loss: 1.3683 - val_accuracy: 0.3231
Epoch 2/25
53600/53600 [==============================] - 4s 82us/sample - loss: 1.3317 - accuracy: 0.3567 - val_loss: 1.2922 - val_accuracy: 0.3847
Epoch 3/25
53600/53600 [==============================] - 5s 86us/sample - loss: 1.2698 - accuracy: 0.3837 - val_loss: 1.2540 - val_accuracy: 0.3912
Epoch 4/25
53600/53600 [==============================] - 4s 83us/sample - loss: 1.2520 - accuracy: 0.3871 - val_loss: 1.2457 - val_accuracy: 0.3955
Epoch 5/25
53600/53600 [==============================] - 5s 86us/sample - loss: 1.2468 - accuracy: 0.3913 - val_loss: 1.2411 - val_accuracy: 0.4000
Epoch 6/25
53600/53600 [==============================] - 5s 86us/sample - loss: 1.2424 - accuracy: 0.3980 - val_loss: 1.2351 - val_accuracy: 0.4041
Epoch 7/25
53600/53600 [==============================]

None

Train on 53600 samples, validate on 13400 samples
Epoch 1/10
53600/53600 [==============================] - 7s 122us/sample - loss: 1.2687 - accuracy: 0.3888 - val_loss: 1.2364 - val_accuracy: 0.4110
Epoch 2/10
53600/53600 [==============================] - 5s 101us/sample - loss: 1.2233 - accuracy: 0.4133 - val_loss: 1.2338 - val_accuracy: 0.4126
Epoch 3/10
53600/53600 [==============================] - 6s 104us/sample - loss: 1.2190 - accuracy: 0.4146 - val_loss: 1.2209 - val_accuracy: 0.4126
Epoch 4/10
53600/53600 [==============================] - 6s 104us/sample - loss: 1.2173 - accuracy: 0.4173 - val_loss: 1.2179 - val_accuracy: 0.4156
Epoch 5/10
53600/53600 [==============================] - 5s 101us/sample - loss: 1.2159 - accuracy: 0.4159 - val_loss: 1.2215 - val_accuracy: 0.4093
Epoch 6/10
53600/53600 [==============================] - 6s 103us/sample - loss: 1.2155 - accuracy: 0.4167 - val_loss: 1.2154 - val_accuracy: 0.4226
Epoch 7/10
53600/53600 [==========================

None

Train on 53600 samples, validate on 13400 samples
Epoch 1/50
53600/53600 [==============================] - 6s 117us/sample - loss: 1.2565 - accuracy: 0.3967 - val_loss: 1.2206 - val_accuracy: 0.4067 0. - ETA: 5s - loss: 1.3053 - accuracy: 0.36 - ETA: 4s - loss: 1.3041  - ETA: 3s - loss: 1.2 - ETA
Epoch 2/50
53600/53600 [==============================] - 5s 102us/sample - loss: 1.2241 - accuracy: 0.4153 - val_loss: 1.2185 - val_accuracy: 0.4188
Epoch 3/50
53600/53600 [==============================] - 6s 108us/sample - loss: 1.2196 - accuracy: 0.4148 - val_loss: 1.2189 - val_accuracy: 0.4184
Epoch 4/50
53600/53600 [==============================] - 6s 107us/sample - loss: 1.2183 - accuracy: 0.4144 - val_loss: 1.2182 - val_accuracy: 0.4125 1.2170 - 
Epoch 5/50
53600/53600 [==============================] - 5s 102us/sample - loss: 1.2175 - accuracy: 0.4168 - val_loss: 1.2237 - val_accuracy: 0.4165
Epoch 6/50
53600/53600 [==============================] - 5s 98us/sample - loss: 1.2169 - a

Example to load old results

In [45]:
import pandas as pd

old_results = pd.read_pickle("./results/config 7 of 7 21-02-19 - 02-12-2019.pkl")


# .style.format({'accuracy': '{:,.3%}'.format})
display(old_results
        .pivot(index='method', columns='config', values='accuracy')
        .style.format('{:,.3%}'))

config,1,2,3,4,5,6,7
method,,,,,,,
KNN-10,49.091%,94.879%,99.242%,100.000%,100.000%,38.848%,34.709%
KNN-100,47.273%,92.364%,98.636%,100.000%,100.000%,42.273%,37.742%
KNN-5,44.848%,95.121%,99.364%,100.000%,100.000%,35.727%,33.194%
KNN-50,53.939%,93.455%,98.879%,100.000%,100.000%,41.030%,37.406%
LDA,55.758%,96.818%,99.970%,100.000%,100.000%,45.242%,39.942%
Logit,56.364%,96.818%,99.879%,100.000%,100.000%,45.667%,40.030%
Naive Bayes,53.333%,96.848%,99.970%,100.000%,100.000%,45.061%,39.945%
Net 30-30-30-30 E10,52.121%,96.364%,99.697%,100.000%,100.000%,45.848%,41.185%
Net 4 E25,33.939%,95.515%,99.242%,100.000%,100.000%,44.727%,41.042%
